# 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

Cloud shell에서 아래 명령어를 입력해 주세요.          

- $ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m        

- $ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

| 평가문항 | 상세기준 |
| --- | --- |
| 1. CSR matrix가 정상적으로 만들어졌다. | 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다. |
| 2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. | 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다. |
| 3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. | MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다. |

## 1. 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [169]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [170]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


- f string 소수점 코드 :.2%

파일이름(ratings)와 겹치지 않도록 컬럼 rating의 이름을 count로 바꿉니다.

In [171]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'ratings':'count'}, inplace=True)

In [172]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [173]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [174]:
# 평점과 제목 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [175]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


# 2. 데이터 분석

In [176]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [177]:
# ratings에 있는 유니크한 영화 제목 수
ratings['title'].nunique()

3628

In [178]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [179]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [180]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

# 3. 내가 선호하는 영화 추가

In [181]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('mission', regex=False)]

,movie_id,title,genre
642,648,Mission: Impossible (1996),Action|Adventure|Mystery
2676,2745,"Mission, The (1986)",Drama
3285,3354,Mission to Mars (2000),Sci-Fi
3554,3623,Mission: Impossible 2 (2000),Action|Thriller
3817,3887,Went to Coney Island on a Mission From God... ...,Drama


In [182]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [1721, 3578, 924 , 1584, 648]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'donghwi'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['donghwi']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,donghwi,1721,5,Titanic (1997)
1,donghwi,3578,5,Gladiator (2000)
2,donghwi,924,5,2001: A Space Odyssey (1968)
3,donghwi,1584,5,Contact (1997)
4,donghwi,648,5,Mission: Impossible (1996)


In [183]:
if not ratings.isin({'user_id':['donghwi']})['user_id'].any():  # user_id에 'donghwi'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,donghwi,1721,5,Titanic (1997)
836479,donghwi,3578,5,Gladiator (2000)
836480,donghwi,924,5,2001: A Space Odyssey (1968)
836481,donghwi,1584,5,Contact (1997)
836482,donghwi,648,5,Mission: Impossible (1996)


# 4. 인덱싱 후 CSR matrix 생성

In [184]:
# 고유한 유저와 영화의 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [185]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,1721,5,27
836479,6039,3578,5,67
836480,6039,924,5,602
836481,6039,1584,5,479


In [186]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. AlternatingLeastSquares 모델 훈련시키기

In [187]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'


In [188]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)


In [189]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [190]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6. 훈련된 모델이 예측한 나의 선호도 파악

In [191]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
donghwi, titanic = user_to_idx['donghwi'], movie_to_idx['Titanic (1997)']
donghwi_vector, titanic_vector = als_model.user_factors[donghwi], als_model.item_factors[titanic]

In [192]:
donghwi_vector   # 나의 벡터

array([ 0.47000462, -0.05217595,  0.07027755, -0.09431639,  0.5325802 ,
        0.79727703, -0.44776565, -0.07964271, -1.5899619 , -0.72929394,
       -0.07775284,  0.42240193, -0.41692418, -0.3896025 , -0.9879204 ,
        0.01830092,  0.40909874,  0.6305792 , -0.481365  ,  0.26164058,
        0.06248047,  1.214257  ,  0.5767754 , -0.4145248 ,  0.08285519,
        0.6415205 ,  0.6008937 , -0.31128588,  0.43888572, -0.5122329 ,
        0.18035786, -0.08299608,  0.2514009 , -0.2977979 , -0.04561334,
        0.1684596 ,  0.87662935, -0.10076124,  0.8223103 , -0.77096456,
        0.3646846 ,  0.04406535,  0.63360214,  0.59032434, -0.7184011 ,
        0.14872634, -0.55275315,  0.65327   , -0.20423362,  0.26718262,
        0.20417012, -0.7101873 , -0.16771486,  0.6474882 , -0.42871964,
       -0.38680297, -0.06313046,  0.7871897 , -0.5922929 , -0.15539756,
       -1.5373075 , -0.48845884,  0.16433045,  0.10572524,  0.5926247 ,
        0.3182412 , -0.37415588, -0.35059685, -0.86603594,  0.33

In [193]:
titanic_vector   # 영화 타이타닉의 벡터

array([ 0.02457456, -0.02373553, -0.00414977,  0.00570934,  0.00401946,
        0.02751422,  0.00999692,  0.01452293, -0.02930282, -0.00478226,
       -0.00802037,  0.03860309,  0.0386439 , -0.01924265, -0.01033675,
        0.00994129,  0.01537809,  0.04104796, -0.02131644,  0.00472953,
       -0.0248108 ,  0.05178101,  0.03749158, -0.03789104,  0.01187153,
        0.00082368,  0.02960778,  0.01492537,  0.0041231 , -0.00489623,
        0.01022932, -0.02698727, -0.01272307, -0.01369261,  0.01407076,
        0.01213607,  0.01976876, -0.00430349,  0.01757392, -0.00956923,
        0.03358164, -0.01359461,  0.03024139,  0.03220628, -0.04526782,
       -0.0307013 ,  0.01108329, -0.0026919 ,  0.00942204,  0.02373183,
        0.01947337, -0.01579277, -0.00181792, -0.00103299, -0.00287274,
        0.00283535,  0.04667271,  0.00076782, -0.01155001,  0.03388461,
       -0.00591128, -0.01171383,  0.00446925,  0.03419684,  0.02607522,
        0.00702316, -0.00560076, -0.01082589, -0.02819487,  0.01

#### 두 벡터를 곱해 1에 가까운 수가 나올 수록 학습이 잘 된 것이다.

In [194]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['Matrix, The (1999)']
matrix_vector = als_model.item_factors[matrix]
np.dot(eunchae_vector, matrix_vector)

-0.0027606871

# 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [195]:
favorite_movie = 'Mission: Impossible (1996)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(58, 1.0),
 (138, 0.48585966),
 (119, 0.42981067),
 (118, 0.41629416),
 (129, 0.41504973),
 (179, 0.40866518),
 (587, 0.40273926),
 (766, 0.39466217),
 (82, 0.39075837),
 (156, 0.38835892),
 (772, 0.38378987),
 (714, 0.3678444),
 (376, 0.35758048),
 (140, 0.35125226),
 (842, 0.34050348)]

In [196]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Mission: Impossible (1996)',
 'True Lies (1994)',
 'Twister (1996)',
 'Con Air (1997)',
 'Conspiracy Theory (1997)',
 'Rock, The (1996)',
 'Batman Returns (1992)',
 'GoldenEye (1995)',
 'Lost World: Jurassic Park, The (1997)',
 'Mask of Zorro, The (1998)',
 'Rising Sun (1993)',
 'Waterworld (1995)',
 'Backdraft (1991)',
 'Clear and Present Danger (1994)',
 'Basic Instinct (1992)']

In [197]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [198]:
get_similar_movie('Gladiator (2000)')

['Gladiator (2000)',
 'Mission: Impossible 2 (2000)',
 'Patriot, The (2000)',
 'Perfect Storm, The (2000)',
 'X-Men (2000)',
 'Shanghai Noon (2000)',
 'U-571 (2000)',
 'Gone in 60 Seconds (2000)',
 'Frequency (2000)',
 'Romeo Must Die (2000)']

In [199]:
get_similar_movie('2001: A Space Odyssey (1968)')

['2001: A Space Odyssey (1968)',
 'Close Encounters of the Third Kind (1977)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'Clockwork Orange, A (1971)',
 'Brazil (1985)',
 '2010 (1984)',
 'Blade Runner (1982)',
 'Alien (1979)',
 'Apocalypse Now (1979)',
 'Aliens (1986)']

# 8. 내가 가장 좋아할 만한 영화 추천

In [200]:
user = user_to_idx['donghwi']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(746, 0.33372873),
 (482, 0.32518402),
 (91, 0.30554077),
 (26, 0.28753597),
 (346, 0.25997016),
 (463, 0.25035003),
 (474, 0.24268025),
 (1537, 0.23806322),
 (224, 0.23792186),
 (317, 0.23040476),
 (1097, 0.22820765),
 (124, 0.2108888),
 (876, 0.21011044),
 (39, 0.20806293),
 (680, 0.20770618),
 (431, 0.2023764),
 (138, 0.2013746),
 (384, 0.19503182),
 (372, 0.19449437),
 (550, 0.19149566)]

In [201]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Perfect Storm, The (2000)',
 'Mission: Impossible 2 (2000)',
 'Close Encounters of the Third Kind (1977)',
 'E.T. the Extra-Terrestrial (1982)',
 'Patriot, The (2000)',
 'Frequency (2000)',
 'Chicken Run (2000)',
 'U-571 (2000)',
 'L.A. Confidential (1997)',
 'Twelve Monkeys (1995)',
 'Brazil (1985)',
 'Matrix, The (1999)',
 '2010 (1984)',
 'Apollo 13 (1995)',
 'Blade Runner (1982)',
 'Gattaca (1997)',
 'True Lies (1994)',
 'Jerry Maguire (1996)',
 'X-Men (2000)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)']

#### explain에 (영화, 기여도) 가 나와있다

In [202]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Blade Runner (1982)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('2001: A Space Odyssey (1968)', 0.13496144168480512),
 ('Contact (1997)', 0.05204960012885875),
 ('Gladiator (2000)', 0.03526577014618581),
 ('Mission: Impossible (1996)', -0.0013126720525140074),
 ('Titanic (1997)', -0.016983221827848435)]

In [203]:
alien = movie_to_idx['Alien (1979)']
explain = als_model.explain(user, csr_data, itemid=alien)
explain

(0.17482634288978954,
 [(602, 0.14759584655991084),
  (67, 0.06633167126834225),
  (58, -0.006159296380020798),
  (479, -0.012666140018497706),
  (27, -0.02027573853994505)],
 (array([[ 0.60383913,  0.13890207,  0.10009203, ...,  0.10027251,
           0.10268542,  0.13822864],
         [ 0.0838745 ,  0.60605496,  0.10195806, ...,  0.11286984,
           0.12291314,  0.1696621 ],
         [ 0.06043948,  0.07569518,  0.61169987, ...,  0.08134389,
           0.08909701,  0.06236173],
         ...,
         [ 0.06054846,  0.08233338,  0.07130251, ...,  0.50076615,
           0.00101885, -0.00563765],
         [ 0.06200547,  0.08875533,  0.0773106 , ...,  0.0591021 ,
           0.51394259,  0.00980942],
         [ 0.08346786,  0.1220248 ,  0.06928067, ...,  0.06092427,
           0.07913921,  0.52604911]]),
  False))

# 회고
1. CSR matrix가 정상적으로 만들어졌다.
- 잘 만들어졌음을 확인할 수 있다.
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
- 기존 추가한 영화와의 선호도는 잘 나왔다.
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
- 영화간 유사도는 의미있지 않았지만, 기여도는 의미있게 측정이 되었다.


- 옛날 영화를 잘 모른다면 장르를 기준으로 판단할 수 밖에 없을 것 같다.
- MF 매트릭스, ALS 모델이 어떻게 구성되는지 공부할 수 있었다.
- 하지만 단지 선호 여부에 따라서만 판단하는 점이 아쉽다.
- 보다 세밀하게 구성되어있는 모델을 공부해보고 싶다.